In [1]:
import os
import pyspark
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"
# Set SPARK_HOME if not already set
os.environ["SPARK_HOME"] = "C:\spark\spark-3.5.5-bin-hadoop3" # Replace with your Spark installation path
os.environ["HADOOP_HOME"] = "C:\hadoop" # Required for Windows
# Check Java installation
java_home = os.getenv("JAVA_HOME")
if java_home is None:
    print("JAVA_HOME environment variable is not set.")
else:
    print(f"JAVA_HOME is set to: {java_home}")
# Check Spark installation
spark_home = os.getenv("SPARK_HOME")
if spark_home is None:
    print("SPARK_HOME environment variable is not set.")
else:
    print(f"SPARK_HOME is set to: {spark_home}")
# Create SparkSession
try:
    spark = (
        SparkSession.builder.appName("quickstart")
        .config("spark.driver.memory", "8g")
        .config("spark.executor.memory", "8g")
        .getOrCreate()
    )
    print("SparkSession created successfully!")
except Exception as e:
    print(f"Error creating SparkSession: {e}")
    

JAVA_HOME is set to: C:\Program Files\Java\jdk-11
SPARK_HOME is set to: C:\spark\spark-3.5.5-bin-hadoop3


<>:6: SyntaxWarning: invalid escape sequence '\P'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\h'
<>:6: SyntaxWarning: invalid escape sequence '\P'
<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\h'
C:\Users\ACER\AppData\Local\Temp\ipykernel_162908\1845603665.py:6: SyntaxWarning: invalid escape sequence '\P'
  os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-11"
C:\Users\ACER\AppData\Local\Temp\ipykernel_162908\1845603665.py:8: SyntaxWarning: invalid escape sequence '\s'
  os.environ["SPARK_HOME"] = "C:\spark\spark-3.5.5-bin-hadoop3" # Replace with your Spark installation path
C:\Users\ACER\AppData\Local\Temp\ipykernel_162908\1845603665.py:9: SyntaxWarning: invalid escape sequence '\h'
  os.environ["HADOOP_HOME"] = "C:\hadoop" # Required for Windows


SparkSession created successfully!


In [2]:
spark = (SparkSession.builder.appName("Datacamp Pyspark Tutorial")
.config("spark.memory.offHeap.enabled","true")
.config("spark.memory.offHeap.size","10g")
.getOrCreate()
)

In [4]:
df = spark.read.csv('data/online_retail.csv',header=True,escape="\"")
df.show(5,0)

+---------+---------+-----------------------------------+--------+----------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate     |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+----------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |01/12/2010 08:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |01/12/2010 08:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |01/12/2010 08:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |01/12/2010 08:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |01/12/2010 08:26|3.39     |17850     |United Kingdom|
+---------+---------+-------------------

In [13]:
# Explanatory data analysis:
print(df.count())

# How many unique customers:
print(df.select('CustomerID').distinct().count())

541909
4373


In [6]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Grouping by Country:
df.groupBy('Country').agg(countDistinct('CustomerID').alias('country_count')).show()

+------------------+-------------+
|           Country|country_count|
+------------------+-------------+
|            Sweden|            8|
|         Singapore|            1|
|           Germany|           95|
|               RSA|            1|
|            France|           87|
|            Greece|            4|
|European Community|            1|
|           Belgium|           25|
|           Finland|           12|
|             Malta|            2|
|       Unspecified|            4|
|             Italy|           15|
|              EIRE|            3|
|         Lithuania|            1|
|            Norway|           10|
|             Spain|           31|
|           Denmark|            9|
|         Hong Kong|            0|
|            Israel|            4|
|           Iceland|            1|
+------------------+-------------+
only showing top 20 rows



In [7]:
df.groupBy('Country').agg(countDistinct('CustomerID').alias('country_count')).orderBy(desc('country_count')).show()

+---------------+-------------+
|        Country|country_count|
+---------------+-------------+
| United Kingdom|         3950|
|        Germany|           95|
|         France|           87|
|          Spain|           31|
|        Belgium|           25|
|    Switzerland|           21|
|       Portugal|           19|
|          Italy|           15|
|        Finland|           12|
|        Austria|           11|
|         Norway|           10|
|        Denmark|            9|
|Channel Islands|            9|
|      Australia|            9|
|    Netherlands|            9|
|         Sweden|            8|
|         Cyprus|            8|
|          Japan|            8|
|         Poland|            6|
|         Greece|            4|
+---------------+-------------+
only showing top 20 rows



In [25]:
# Checking last purchase:
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
df = df.withColumn('date',to_timestamp("InvoiceDate", 'dd/MM/yy HH:mm'))
df.select(max("date")).show()
df.select(min("date")).show()

min_date = df.select(min("date")).head()[0]
print(min_date)

+-------------------+
|          max(date)|
+-------------------+
|2011-12-09 12:50:00|
+-------------------+

+-------------------+
|          min(date)|
+-------------------+
|2010-12-01 08:26:00|
+-------------------+

2010-12-01 08:26:00


In [26]:
df.printSchema()
df.show(5,0)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- date: timestamp (nullable = true)

+---------+---------+-----------------------------------+--------+----------------+---------+----------+--------------+-------------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate     |UnitPrice|CustomerID|Country       |date               |
+---------+---------+-----------------------------------+--------+----------------+---------+----------+--------------+-------------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |01/12/2010 08:26|2.55     |17850     |United Kingdom|2010-12-01 08:26:00|
|536365   |71053    |WHITE METAL LANTERN                |6       

DataFrame[min(date): timestamp]
